# Université Paul Sabatier

M1IAFA - Recherche d'information

**TP 5**

Enseignants : Lynda Tamine et Jesús Lovón

Notebook proposé par : José G. Moreno


2024

---

💡 Penser à développer des scripts et fonctions auxiliaires qui vont permettre réutiliser les commandes récurrentes dans ce TP et les prochains. Ceci aussi vous permettra de garder de bon pratique du code et faciliter vos débogages.

---


### Attention ❗❗ Pour la note du TP :
🚨 *Questions de code* : Remplir le code manquant dans les parties correspondantes (le code commenté bénéficie des meilleures notes).

🚨 *Questions ouvertes* : Ecrivez votre réponse textuelle sous forme de commentaires dans les cellules correspondantes.

🚨 *Laissez vos sorties* pour les cellules où vous écrivez le code. Les sorties vides (notebook ou cellules non exécutées) correspondent à 0 points.

---

# TP 5. PyTerrier - Neural Re-Ranking

Dans ce TP vous allez :

 - reclasser des documents en utilisant des modèles neuronaux comme KNRM, Vanilla BERT, EPIC et monoT5.


# 0. Installation


In [ ]:
!pip install -q --upgrade python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py

# Installation des plugins Pyterrier  

Nous installons les plugins PyTerrier [OpenNIR](https://opennir.net/) et [monoT5](https://github.com/terrierteam/pyterrier_t5). Vous pouvez ignorer sans risque les erreurs de version des paquets.

In [ ]:
!pip install -q --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.3/114.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# 1. Preliminary steps

In [ ]:
import os
import tensorflow as tf

**[PyTerrier](https://github.com/terrier-org/pyterrier) initialization**

In [ ]:
import pyterrier as pt
if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')


from pyterrier.measures import * # allow for natural measure names
import onir_pt

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# 2. Indexation

In [ ]:
!rm -rf ./terrier_cord19/*

In [ ]:
import os

pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(),
                              fields=('abstract',),
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

[INFO] [starting] building docstore
[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
docs_iter:   0%|                                    | 0/192509 s<?, ?doc/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 0.00/269M s<?, ?B/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 123k/269M s<03:56, 1.14MB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.4%| 1.03M/269M s<59.01s, 4.54MB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 2.2%| 5.97M/269M s<14.43s, 18.2MB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazon

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

<ipython-input-6-47146416b637>:11: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(cord19.get_corpus_iter(),


20:04:10.116 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
20:05:55.697 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Inde

# 3. Re-Rankers

Commençons à explorer quelques méthodes neuronales de re-classement ! Nous pouvons les construire à partir de zéro en utilisant `onir_pt.reranker`.

Le modèle de re-ranking d'OpenNIR est composé de :
 - `ranker` (par exemple, `drmm`, `knrm`, ou `pacrr`). Ceci définit l'architecture neuronale de classement.
 - `vocab` (par exemple, `wordvec_hash`, ou `bert`). Ceci définit comment le texte est encodé par le modèle. Cette approche rend facile l'échange de différentes représentations de texte.

L'exécution de cette ligne prendra quelques minutes car elle télécharge et prépare les vecteurs de mots.

In [ ]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

config file not found: config
[2024-04-19 20:06:16,620][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2024-04-19 20:06:24,152][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2024-04-19 20:06:24,161][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [7.34s] [682M] [129MB/s]
[2024-04-19 20:06:24,172][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [7.55s]
[2024-04-19 20:06:24,173][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2024-04-19 20:06:46,752][WordvecHashVocab][DEBUG] [finished] extracting vecs [22.58s]
[2024-04-19 20:06:46,753][WordvecHashVocab][DEBUG] [starting] loading vecs into memory
[2024-04-19 20:09:40,422][WordvecHashVocab][DEBUG] [finished] loading vecs into memory [02:54]
[2024-04-19 20:09:40,703][WordvecHashVocab][DEBUG] [starting] writing

Voyons comment ces modèles fonctionnent pour le classement !

In [ ]:
tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF") % 50
get_text = pt.text.get_text(cord19, 'abstract') #>> pt.apply.title_abstract(lambda r: r['title'] + ' ' + r['abstract'])


In [ ]:
topics = cord19.get_topics(variant='description')
qrels = cord19.get_qrels()

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: s] [18.7kB] [23.8MB/s]
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: s] [1.14MB] [3.64MB/s]


In [ ]:
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)


test_qrels = qrels # seulement les annotations des topics en réponse sont utilisés, donc pas de problème si on utilise tout
train_qrels = qrels
valid_qrels = qrels

In [ ]:

# build a sub-pipeline to get the concatenated title and abstract text
pipeline = tfidf >> get_text >> knrm
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2024-04-19 20:10:18,043][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2024-04-19 20:10:18,054][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2024-04-19 20:10:19,115][onir_pt][DEBUG] [finished] batches: [1.06s] [188it] [177.38it/s]


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10
0,TFIDF,0.054568,0.123589,0.595818,0.546667
1,TFIDF >> KNRM,0.042547,0.112703,0.423958,0.360000


Cela ne fonctionne pas très bien car le modèle n'est pas entraîné ; il utilise des poids aléatoires pour combiner les scores de la matrice de similarité.

# 4. Entraînement du re-ranker

Vous pouvez entraîner des modèles de re-classement dans PyTerrier en utilisant la méthode `fit`.

In [ ]:
pipeline.fit(
    train_topics,
    train_qrels,
    valid_topics,
    valid_qrels)

[2024-04-19 20:10:23,759][onir_pt][DEBUG] [starting] validation
[2024-04-19 20:10:23,761][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2024-04-19 20:10:23,763][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/63 s<?, ?it/s]

[2024-04-19 20:10:24,270][onir_pt][DEBUG] [finished] batches: s] [63it] [124.46it/s]
[2024-04-19 20:10:24,558][onir_pt][DEBUG] [finished] validation s]
[2024-04-19 20:10:24,564][onir_pt][INFO] pre-validation: 0.0053
[2024-04-19 20:10:24,573][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2024-04-19 20:10:24,575][onir_pt][DEBUG] [starting] training
[2024-04-19 20:10:24,576][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2024-04-19 20:10:25,051][onir_pt][DEBUG] not enough negs
[2024-04-19 20:10:26,209][onir_pt][DEBUG] not enough negs
[2024-04-19 20:10:28,349][onir_pt][DEBUG] not enough negs
[2024-04-19 20:10:30,401][onir_pt][DEBUG] not enough negs
[2024-04-19 20:10:31,277][onir_pt][DEBUG] not enough negs
[2024-04-19 20:10:33,194][onir_pt][DEBUG] not enough negs
[2024-04-19 20:10:35,148][onir_pt][DEBUG] [finished] train pairs: [10.57s] [1024it] [96.86it/s]
[2024-04-19 20:10:35,155][onir_pt][DEBUG] [finished] training [10.58s]
[2024-04-19 20:10:35,156][onir_pt][INFO] training   it=0 loss=0.2478
[2024-04-19 20:10:35,157][onir_pt][DEBUG] [starting] validation
[2024-04-19 20:10:35,159][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2024-04-19 20:10:35,160][onir_pt][DEBUG] [starting] batches


...

[2024-04-19 20:26:42,735][onir_pt][DEBUG] [finished] batches: s] [63it] [112.90it/s]
[2024-04-19 20:26:43,038][onir_pt][DEBUG] [finished] validation s]
[2024-04-19 20:26:43,040][onir_pt][INFO] validation it=79 map=0.0059 ndcg=0.0153 P_10=0.0720
[2024-04-19 20:26:43,040][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2024-04-19 20:26:43,042][onir_pt][DEBUG] [starting] training
[2024-04-19 20:26:43,042][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2024-04-19 20:26:43,256][onir_pt][DEBUG] not enough negs
[2024-04-19 20:26:43,283][onir_pt][DEBUG] not enough negs
[2024-04-19 20:26:43,813][onir_pt][DEBUG] not enough negs
[2024-04-19 20:26:45,945][onir_pt][DEBUG] not enough negs
[2024-04-19 20:26:50,447][onir_pt][DEBUG] not enough negs
[2024-04-19 20:26:51,195][onir_pt][DEBUG] not enough negs
[2024-04-19 20:26:52,097][onir_pt][DEBUG] not enough negs
[2024-04-19 20:26:53,691][onir_pt][DEBUG] not enough negs
[2024-04-19 20:26:54,374][onir_pt][DEBUG] [finished] train pairs: [11.33s] [1024it] [90.39it/s]
[2024-04-19 20:26:54,379][onir_pt][DEBUG] [finished] training [11.34s]
[2024-04-19 20:26:54,379][onir_pt][INFO] training   it=80 loss=0.2188
[2024-04-19 20:26:54,380][onir_pt][DEBUG] [starting] validation
[2024-04-19 20:26:54,381][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2024-04-19 20:26:54,382][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/63 s<?, ?it/s]

[2024-04-19 20:26:54,935][onir_pt][DEBUG] [finished] batches: s] [63it] [114.17it/s]
[2024-04-19 20:26:55,232][onir_pt][DEBUG] [finished] validation s]
[2024-04-19 20:26:55,235][onir_pt][INFO] validation it=80 map=0.0060 ndcg=0.0154 P_10=0.0720
[2024-04-19 20:26:55,236][onir_pt][INFO] early stopping; model reverting back to it=60


In [ ]:
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM (trained)'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2024-04-19 20:26:58,921][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2024-04-19 20:26:58,923][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2024-04-19 20:27:00,596][onir_pt][DEBUG] [finished] batches: [1.67s] [188it] [112.48it/s]


,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10
0,TFIDF,0.054568,0.123589,0.595818,0.546667
1,TFIDF >> KNRM (trained),0.051524,0.121720,0.513901,0.433333


Le résultat est plus élévé, mais cela reste moins bon que le TFIDF. Proposez une hypothèse sur le problème.

L'hypothèse la plus probable serait que la performance inférieure de KNRM par rapport à TF-IDF c'est à de manque de données d'entraînement pour le KNRM. Le KNRM nécessite une bonne taille de données d'entraînement pour capturer les associations entre les termes de la requête et les termes du document de manière plus pertinente.

# 5. Vanilla BERT

Les modèles linguistiques contextualisés, tels que [BERT](https://arxiv.org/abs/1810.04805), sont des modèles neuronaux beaucoup plus puissants qui se sont avérés efficaces pour le classement.

Nous allons essayer d'utiliser une version "vanille" (ou "mono") du modèle BERT. Le modèle BERT est pré-entraîné pour la modélisation du langage et la prédiction de la phrase suivante.

In [ ]:
#del knrm # clear out memory from KNRM
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='abstract', vocab_config={'train': True})

100%|██████████| 231508/231508 s<0ms, 1270546.20B/s]
100%|██████████| 433/433 s<0ms, 276638.79B/s]
100%|██████████| 440473133/440473133 [13.29s<0ms, 33147949.46B/s] 


Voyons comment ce modèle se comporte sur la collection TREC COVID.

In [ ]:
pipeline = tfidf % 50 >> get_text >> vbert
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> VBERT'],
    baseline=0,
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

[2024-04-19 20:27:36,079][onir_pt][ERROR] gpu=True, but CUDA is not available. Falling back on CPU.
[2024-04-19 20:27:36,086][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/188 s<?, ?it/s]

[2024-04-19 20:46:32,135][onir_pt][DEBUG] [finished] batches: [18:56] [188it] [ 6.04s/it]


,name,nDCG,nDCG@10,AP(rel=2),P(rel=2)@10,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value,AP(rel=2) +,AP(rel=2) -,AP(rel=2) p-value,P(rel=2)@10 +,P(rel=2)@10 -,P(rel=2)@10 p-value
0,TFIDF,0.123589,0.595818,0.054568,0.546667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TFIDF >> VBERT,0.118277,0.479929,0.049198,0.420000,3.0,12.0,0.11542,4.0,11.0,0.02287,3.0,12.0,0.012239,3.0,12.0,0.024362


Comme nous le constatons, bien que le modèle soit pré-entraîné, il n'obtient pas de très bons résultats. Cela est dû au fait qu'il n'est pas réglé pour la tâche de classement par pertinence.

Cependant, nous pouvons entraîner le modèle pour le classement (comme indiqué ci-dessus pour KNRM).

# 6. monoT5

Le modèle [monoT5](https://arxiv.org/abs/2003.06713) évalue les documents à l'aide d'un modèle de langage causal. Voyons comment cette approche fonctionne sur TREC COVID.

La classe `MonoT5ReRanker` de `pyterrier_t5` charge automatiquement une version du classeur monoT5 qui est entraînée sur le jeu de données MS MARCO passage.

In [ ]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k s<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M s<?, ?B/s]

config.json:   0%|          | 0.00/1.21k s<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.84k s<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M s<?, ?B/s]

In [ ]:
pipeline = (tfidf >> get_text >> monoT5)
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> T5'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10, "mrt"]
)

monoT5:   0%|          | 0/188 s<?, ?batches/s]

,name,AP(rel=2),nDCG,nDCG@10,P(rel=2)@10,mrt
0,TFIDF,0.054568,0.123589,0.595818,0.546667,75.879755
1,TFIDF >> T5,0.058684,0.128119,0.685513,0.600000,78086.572887


Comme on pouvait s'y attendre, les résultats sont bien meilleurs en termes de NDCG@10 (0.5958 vs 0.6855).

# Tâche pratique

Comme pour le TP precedent, utilisez les modèles implémentés pour cord19 dans une tâche de question-réponse. Dans ce contexte, les requêtes sont de questions et les documents sont des documents qui pourraient contenir la réponse. Notez que vous devez refaire l'indexation ainsi que les autres étapes étudiées dans ce TP. Vous pouvez le télécharger le dataset en utilisant les lines de code ci-dessous.

In [ ]:
fiqa = {}
fiqa['train'] = pt.datasets.get_dataset('irds:beir/fiqa/train')
fiqa['valid'] = pt.datasets.get_dataset('irds:beir/fiqa/dev')
fiqa['test'] = pt.datasets.get_dataset('irds:beir/fiqa/test')

test_topics = fiqa['test'].get_topics(variant='text')
test_qrels = fiqa['test'].get_qrels()

train_topics = fiqa['train'].get_topics(variant='text')
train_qrels = fiqa['train'].get_qrels()

valid_topics = fiqa['valid'].get_topics(variant='text')
valid_qrels = fiqa['valid'].get_qrels()

[INFO] [starting] opening zip file
[INFO] If you have a local copy of https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/17918ed23cd04fb15047f73e6c3bd9d9
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip
[INFO] [finished] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip: [2.06s] [17.9MB] [8.69MB/s]
[INFO] [finished] opening zip file [3.08s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]


In [ ]:
print(test_qrels)

        qid   docno  label iteration
0         8  566392      1         0
1         8   65404      1         0
2        15  325273      1         0
3        18   88124      1         0
4        26  285255      1         0
...     ...     ...    ...       ...
1701  11039  330058      1         0
1702  11039   91183      1         0
1703  11054  155053      1         0
1704  11054  321015      1         0
1705  11088  437100      1         0

[1706 rows x 4 columns]


In [ ]:
!pip install -q --upgrade fastrank lightgbm==3.1.1
!pip install -q python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.2 MB/s eta 0:00:00


In [ ]:
def append_baseline(system, baseline, max_results = 1000):
    return (system ^ baseline) % max_results

In [ ]:
tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF")
ltr_feats1 = (tfidf % RANK_CUTOFF) >> pt.text.get_text(cord19, ["title", "date", "doi"]) >> (
    pt.transformer.IdentityTransformer()
    ** # score of text for query 'coronavirus covid'
    (pt.apply.query(lambda row: 'coronavirus covid') >> tfidf)
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="title", takes='docs', wmodel='TF_IDF') )
    ** # date 2020
    (pt.apply.doc_score(lambda row: int("2020" in row["date"])))
    ** # has doi
    (pt.apply.doc_score(lambda row: int( row["doi"] is not None and len(row["doi"]) > 0) ))
    ** # abstract coordinate match
    pt.BatchRetrieve(index_ref, wmodel="CoordinateMatch")
)

In [ ]:
import fastrank

train_request = fastrank.TrainRequest.coordinate_ascent()

params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(train_request, form='fastrank')

%time ca_pipe.fit(train_topics, train_qrels)



21:29:56.101 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.10/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7b562ef6d0f0>, expected 54959 received 55321, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=42, n_jobs=2)

rf_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(rf, form='regression')

%time rf_pipe.fit(train_topics, train_qrels)



In [ ]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

lmart_x_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[10]})

%time lmart_x_pipe.fit(train_topics, train_qrels, valid_topics, valid_qrels)

In [ ]:
#réponse TFIDF + CA avec TFIDF + RF
ca_cf = append_baseline(ca_pipe, rf_pipe)
ca_lm = append_baseline(ca_pipe, lmart_x_pipe)
lm_cf = append_baseline(lmart_x_pipe, rf_pipe)

In [ ]:
q1 = "Why are big companies like Apple or Google not included in the Dow Jones Industrial Average (DJIA) index?"
q1 = q1.split()
q1 = [token for token in q1 if token[0].isupper()]
print(ca_pipe.transform(q1))
print(rf_pipe.transform(q1))
print(lmart_x_pipe.transform(q1))

**Question**. Identifiez-vous des avantages par rapport à l'utilisation de Learning to Rank ?

Le temps d'exécution devrait être plus rapide car il n'y a pas d'étape de formation de modèle dans cette pipeline, contrairement à LTR. De plus, il n'est pas nécessaire d'annoter les données avec des scores de pertinence, car les modèles de recherche utilisés ici (TF-IDF, KNRM, VBERT, T5) peuvent être directement utilisés pour le classement des résultats. Cependant, il est important de noter que mon ordinateur n'est pas très puissant, ce qui a limité mes capacités à effectuer des tests approfondis, y compris pour la dernière partie du TP4. Mesage d'erreur de collab(Votre session a planté après avoir utilisé toute la mémoire RAM disponible.)